## 🧬 What Pharmit Does When You Upload an .sdf with Multiple Molecules

When you click “Load Features” (or upload an .sdf or .mol2 file with multiple molecules), Pharmit only uses the first molecule in the file to:

Generate the pharmacophore

Display the 3D structure

Create shape-based searches

The other molecules are ignored.

## 💡 Why This Happens

Pharmit is designed for interactive pharmacophore-based virtual screening, not multi-ligand consensus modeling.
Its workflow assumes:

One ligand → generate pharmacophore → search databases for similar molecules.

So even if your .sdf has 1000 entries, Pharmit’s backend loads only the first conformer / first molecule record to:

Calculate hydrogen bond donors/acceptors, hydrophobics, aromatics, etc.

Display the molecular surface and features.

There’s currently no built-in function in Pharmit to merge or average features across multiple hits.

### INSTEAD - First Generate consensus pharmacophore model with RDKIT:

this version of the RDKit script will take an .sdf file with many molecules (e.g. 1000 hits) and build a consensus pharmacophore that combines the common features across all of them.

It automatically:

Embeds + aligns all molecules in 3D

Detects features (H-bond donors, acceptors, aromatics, hydrophobics)

Clusters nearby features spatially to merge them into consensus points

Outputs a Pharmit-compatible .phar file

## 🧬 RDKit Consensus Pharmacophore Builder (for multi-hit .sdf)

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign, ChemicalFeatures
from rdkit import RDConfig
import numpy as np
from sklearn.cluster import DBSCAN
import json, os

# === Input / Output ===
input_sdf = "VinaResults_9ety_apo_hit_ligandsSite2.sdf"
output_phar = "consensus.phar"

# === Load molecules ===
mols = [m for m in Chem.SDMolSupplier(input_sdf) if m is not None]
print(f"Loaded {len(mols)} molecules")

# === Generate 3D conformers (if missing) ===
for m in mols:
    if not m.GetNumConformers():
        m = Chem.AddHs(m)
        AllChem.EmbedMolecule(m, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(m)

# === Align all molecules to the first ===
ref = mols[0]
for m in mols[1:]:
    try:
        rdMolAlign.AlignMol(m, ref)
    except:
        pass

# === Set up RDKit feature factory ===
fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

# === Collect all feature positions and types ===
positions, families = [], []
for m in mols:
    feats = factory.GetFeaturesForMol(m)
    for f in feats:
        fam = f.GetFamily()
        pos = np.array(f.GetPos())
        positions.append(pos)
        families.append(fam)

positions = np.array(positions)
families = np.array(families)

# === Cluster features of same type (DBSCAN groups close ones) ===
consensus_features = []
eps = 1.5  # Å radius for clustering
min_samples = max(2, len(mols)//20)  # feature must appear in at least 5% of molecules

for fam in np.unique(families):
    fam_positions = positions[families == fam]
    if len(fam_positions) == 0:
        continue
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(fam_positions)
    labels = clustering.labels_

    for label in set(labels):
        if label == -1:
            continue  # ignore outliers
        cluster_points = fam_positions[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        consensus_features.append({
            "family": fam,
            "x": float(centroid[0]),
            "y": float(centroid[1]),
            "z": float(centroid[2]),
            "count": int(len(cluster_points)),
            "radius": round(eps, 2)
        })

print(f"✅ Found {len(consensus_features)} consensus pharmacophore points")

# === Convert to Pharmit-compatible JSON ===
pharmit_format = {
    "points": [
        {
            "enabled": True,
            "shape": f["family"],
            "radius": f["radius"],
            "location": {
                "x": f["x"],
                "y": f["y"],
                "z": f["z"]
            }
        }
        for f in consensus_features
    ],
    "excluded": []
}

output_json = output_phar.replace(".phar", ".json")
with open(output_json, "w") as f:
    json.dump(pharmit_format, f, indent=2)

print(f"✅ Saved Pharmit-compatible pharmacophore to {output_json}")



[19:14:42] Explicit valence for atom # 0 N, 4, is greater than permitted
[19:14:42] ERROR: Could not sanitize molecule ending on line 1898
[19:14:42] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
[19:14:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[19:14:42] ERROR: Could not sanitize molecule ending on line 2432
[19:14:42] ERROR: Explicit valence for atom # 3 N, 4, is greater than permitted
[19:14:42] Explicit valence for atom # 16 N, 4, is greater than permitted
[19:14:42] ERROR: Could not sanitize molecule ending on line 2703
[19:14:42] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted
[19:14:42] Explicit valence for atom # 5 N, 4, is greater than permitted
[19:14:42] ERROR: Could not sanitize molecule ending on line 3292
[19:14:42] ERROR: Explicit valence for atom # 5 N, 4, is greater than permitted
[19:14:42] Explicit valence for atom # 16 N, 4, is greater than permitted
[19:14:42] ERROR: Could not sanitize molecule en

Loaded 700 molecules
✅ Found 7 consensus pharmacophore points
✅ Saved Pharmit-compatible pharmacophore to consensus.json


## Using the CONPHAR library - https://doi.org/10.1021/acs.jcim.3c01439